In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.signal import butter, filtfilt, medfilt, savgol_filter

In [12]:
repo_root = Path.cwd() / "ingested_data" # current working directory, then go up one
csv_file = repo_root / "sensors_testRide_fused_20251129_215710.csv"
# 2. Load CSV
df = pd.read_csv(csv_file)

In [13]:
# Quick peek
print(df.columns)

Index(['master_time', 't_rel', 'Accelerometer_z', 'Accelerometer_y',
       'Accelerometer_x', 'TotalAcceleration_z', 'TotalAcceleration_y',
       'TotalAcceleration_x', 'Compass_magneticBearing', 'Gravity_z',
       'Gravity_y', 'Gravity_x', 'Gyroscope_z', 'Gyroscope_y', 'Gyroscope_x',
       'Location_bearingAccuracy', 'Location_speedAccuracy',
       'Location_verticalAccuracy', 'Location_horizontalAccuracy',
       'Location_speed', 'Location_bearing', 'Location_altitude',
       'Location_longitude', 'Location_latitude', 'Orientation_qz',
       'Orientation_qy', 'Orientation_qx', 'Orientation_qw',
       'Orientation_roll', 'Orientation_pitch', 'Orientation_yaw'],
      dtype='object')


In [14]:
def apply_filters(df, columns, method="butter", params=None, replace=False):
    """
    Apply smoothing filters to selected dataframe columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe.
    columns : list of str
        Column names to filter.
    method : str
        Filtering method. Options:
        - "butter" : Butterworth low-pass
        - "median" : Median filter
        - "savgol" : Savitzky-Golay filter
        - "rolling" : Rolling mean
    params : dict
        Parameters for the chosen filter.
        Examples:
        - butter: {"cutoff": 5, "fs": 50, "order": 2}
        - median: {"kernel_size": 3}
        - savgol: {"window_length": 11, "polyorder": 2}
        - rolling: {"window": 5}
    replace : bool
        If True, overwrite the original column with smoothed values.
        If False, add new column as "smooth_<col>".

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with smoothed columns added or replaced.
    """
    if params is None:
        params = {}

    for col in columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in dataframe.")

        x = df[col].to_numpy()

        if method == "butter":
            cutoff = params.get("cutoff", 5)
            fs = params.get("fs", 50)
            order = params.get("order", 2)
            nyq = 0.5 * fs
            Wn = cutoff / nyq
            b, a = butter(order, Wn, btype="low")
            y = filtfilt(b, a, x)

        elif method == "median":
            k = params.get("kernel_size", 3)
            y = medfilt(x, kernel_size=k)

        elif method == "savgol":
            wl = params.get("window_length", 11)
            po = params.get("polyorder", 2)
            y = savgol_filter(x, wl, po)

        elif method == "rolling":
            w = params.get("window", 5)
            y = pd.Series(x).rolling(window=w, min_periods=1).mean().to_numpy()

        else:
            raise ValueError(f"Unknown method '{method}'.")

        if replace:
            df[col] = y
        else:
            df[f"smooth_{col}"] = y

    return df

In [27]:
import matplotlib.pyplot as plt

def plot_multiple_columns(df, columns, colors=None, time_col='master_time'):
    """
    Produce separate plots for each column in 'columns'.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe containing the data.
    columns : list of str
        List of column names to plot.
    colors : list of str or None
        List of colors for each plot. If None, defaults to matplotlib cycle.
    time_col : str
        Column to use for x-axis (default 'master_time').
    """
    if colors is None:
        colors = [None] * len(columns)  # fallback to default cycle

    if len(colors) != len(columns):
        raise ValueError("Length of colors list must match length of columns list.")

    for col, color in zip(columns, colors):
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in dataframe.")

        plt.figure(figsize=(10, 4))
        plt.plot(df[time_col], df[col], color=color)
        plt.title(f"{col}")
        plt.xlabel(time_col)
        plt.ylabel("Value")
        plt.grid(True)  # always on
        plt.show()


In [28]:
import plotly.express as px

fig = px.scatter_map(df, lat="Location_latitude", lon="Location_longitude", zoom=12)
fig.update_layout(mapbox_style="open-street-map")

# Save static image
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed